# ENTREGABLE 1

## 1. Configuración del entorno de trabajo

### Consigna

- Instalar y configurar un sistema de gestión de bases de datos relacional (PostgreSQL o SQL Server).
- Crear una base de datos de trabajo y preparar el entorno para futuras transformaciones (conexión desde Python vía ORM recomendado).

### Detalle de implementación

Se configuró un entorno de base de datos con Docker utilizando **PostgreSQL**. Este servicio ejecuta PostgreSQL y monta los scripts SQL necesarios para crear las tablas al iniciar. Además, se incluye **Adminer** para gestionar la base de datos de forma gráfica.

El entorno está listo para interactuar con la base de datos desde Python usando **SQLAlchemy** como ORM, lo que facilita la implementación de futuras transformaciones y expansiones del proyecto.

---

## 2. Carga inicial de datos

### Consigna

- Crear tablas e importar datos a partir de los archivos CSV proporcionados.
- Ajustar los tipos de datos según corresponda para preservar la integridad semántica y evitar errores de procesamiento.

### Detalle de implementación

La implementación se realizó en **dos fases**:

### Fase 1: Creación de modelos de tablas e importación de datos

Se crearon modelos de tablas basados en las estructuras de los scripts SQL provistos. En esta fase inicial, no se aplicaron restricciones complejas, sino que se enfocó en asegurar la correcta relación entre tablas mediante claves foráneas y los tipos de datos básicos. El objetivo fue importar los datos desde los archivos CSV y asegurar que las claves foráneas (relación entre tablas) estuvieran correctamente configuradas.

**Ejemplo de implementación:**

```python
class Product(BaseModel):

    __tablename__ = TableNames.PRODUCTS

    product_id: Mapped[int] = mapped_column(ProductColumns.PRODUCT_ID, primary_key=True)
    name: Mapped[str] = mapped_column(ProductColumns.NAME, String)
    description: Mapped[str] = mapped_column(ProductColumns.DESCRIPTION, String)
    price: Mapped[Decimal] = mapped_column(ProductColumns.PRICE, Numeric)
    stock: Mapped[int] = mapped_column(ProductColumns.STOCK, Integer)
    category_id: Mapped[int] = mapped_column(
        CategoryColumns.CATEGORY_ID, ForeignKey("Categorias.CategoriaID"), nullable=False
    )
    category: Mapped["Category"] = relationship("Category", backref="products")

    @classmethod
    def from_dict(cls, data: dict):
        return cls(
            product_id=data.get("product_id"),
            name=data.get("name", "").strip(),
            description=data.get("description", "").strip(),
            price=Decimal(data.get("price", 0)),
            stock=data.get("stock", 0),
            category_id=data.get("category_id"),
        )
```
### Fase  2: Ajustes de tipos de datos y restricciones de integridad

Una vez que los datos fueron importados correctamente, se modificaron los modelos para asegurar la integridad semántica de los datos. Esto incluyó:

- Ajuste de tipos de datos: Se cambiaron algunas columnas para asegurar que almacenaran los datos en el tipo adecuado (por ejemplo, fechas o cadenas con longitud fija).

- Aplicación de restricciones de integridad: Se añadieron restricciones como CHECK y UNIQUE para evitar que los datos violaran las reglas de negocio. Esto garantiza que los datos sean válidos y coherentes con las expectativas del sistema.

**Ejemplo de implementación final (con restricciones):**

```python


class Product(BaseModel):

    __tablename__ = TableNames.PRODUCTS

    __table_args__ = (
        CheckConstraint(
            f"{ProductColumns.STOCK} >= 0", name="stock_non_negative"),
        CheckConstraint(
            f"{ProductColumns.PRICE} >= 0", name="price_non_negative"),
    )

    product_id: Mapped[int] = mapped_column(
        ProductColumns.PRODUCT_ID, primary_key=True)
    name: Mapped[str] = mapped_column(ProductColumns.NAME, String(255))
    description: Mapped[str] = mapped_column(
        ProductColumns.DESCRIPTION, String)
    price: Mapped[Decimal] = mapped_column(ProductColumns.PRICE, Numeric)
    stock: Mapped[int] = mapped_column(ProductColumns.STOCK, Integer)
    category_id: Mapped[int] = mapped_column(
        CategoryColumns.CATEGORY_ID,
        ForeignKey("Categorias.CategoriaID"),
        nullable=False,
    )
    category: Mapped["Category"] = relationship("Category", backref="products")

    @classmethod
    def from_dict(cls, data: dict):
        return cls(
            product_id=data.get("product_id"),
            name=data.get("name", "").strip(),
            description=data.get("description", "").strip(),
            price=Decimal(data.get("price", 0)),
            stock=data.get("stock", 0),
            category_id=data.get("category_id"),
        )

```

---

## 3. Tratamiento de campos semi-estructurados


### Consigna

- Identificar columnas que contengan datos en formatos como JSON, listas, o concatenaciones delimitadas.

- Aplicar técnicas de limpieza y transformación para estructurar adecuadamente la información.


### Identificación de columnas con datos semiestructurados

| **Tabla.Columna**               | **Descripción y ejemplo**                                                       | **Decisión de transformación**                                                                                                                                                       |
|----------------------------------|---------------------------------------------------------------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **Categorias.Descripción**       | Es una descripción con ejemplos de la misma separados por comas, ej: 'Teléfonos, computadoras, accesorios y dispositivos inteligentes' | No se normaliza, dado que parece una descripción y las preguntas de negocio no requieren ese nivel de detalle. Mantiene su formato como texto.                                        |
| **MetodosPago.Descripcion**     | Caso similar al de `Categorias.Descripción`, ej: 'Pago con tarjeta de crédito VISA, MasterCard o American Express'                  | No se aplicó transformación, dado que no se requiere conocer marcas de tarjetas, solo la modalidad de pago (ej., 'Tarjeta de Crédito', 'Transferencia Bancaria').                   |

---

## 4.Análisis exploratorio y evaluación de calidad de datos

### Consigna


- Explorar la estructura y el contenido de los datos utilizando consultas SQL.

- Implementar análisis exploratorio complementario en Python mediante un ORM (como SQLAlchemy o psycopg2).

- Detectar valores nulos, duplicados, atípicos y otras inconsistencias.

- Identificar claves primarias y foráneas implícitas, atributos principales y variables relevantes para el negocio.

- Proponer y documentar acciones de preprocesamiento y corrección de calidad de datos.

### Desarrollo de consignas 

#### 1. Explorar la estructura y el contenido de los datos utilizando consultas SQL
En esta fase del análisis, el objetivo principal es obtener una comprensión detallada sobre la estructura y el contenido de las tablas dentro de la base de datos, así como la naturaleza de los datos. Este paso es fundamental porque nos permite identificar las relaciones entre las tablas, la calidad de los datos, y cualquier posible inconsistencia que pueda requerir correcciones antes de realizar análisis más avanzados.


- Consultas de exploración

Consulta para identificar tablas

In [1]:
from app.utils import postgres_utils

query = """
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'public';
"""
tables = postgres_utils.run_query(query)

tables

,table_name
0,Categorias
1,Productos
2,Usuarios
3,Ordenes
4,DetalleOrdenes
5,DireccionesEnvio
6,Carrito
7,OrdenesMetodosPago
8,MetodosPago
9,ReseñasProductos


* Consulta de la estructura de las tablas , para conocer columnas y tipos de datos de las mismas.

In [2]:

from app.utils import postgres_utils

query_tables_structure = """
        SELECT table_name, column_name, data_type
        FROM information_schema.columns
        WHERE table_schema = 'public'
        ORDER BY table_name, ordinal_position;
    """

postgres_utils.run_query(query_tables_structure)

,table_name,column_name,data_type
0,Carrito,CarritoID,integer
1,Carrito,UsuarioID,integer
2,Carrito,ProductoID,integer
3,Carrito,Cantidad,integer
4,Carrito,FechaAgregado,timestamp without time zone
5,Categorias,CategoriaID,integer
6,Categorias,Nombre,character varying
7,Categorias,Descripcion,character varying
8,DetalleOrdenes,DetalleID,integer
9,DetalleOrdenes,OrdenID,integer


Consulta para obtener primeros registros de cada tabla, para explorar los datos y formatos:

In [3]:
from app.utils import postgres_utils
import logging

logging.getLogger("sqlalchemy.engine").setLevel(logging.WARNING)

def get_first_records(table_name):
    query = f"SELECT * FROM \"{table_name}\" LIMIT 10;"
    data = postgres_utils.run_query(query)
    return data

tables_list = tables['table_name'].unique()
for table in tables_list:
    data= postgres_utils.run_query(f"SELECT * FROM \"{table}\" LIMIT 5;")
    display(data)


,CategoriaID,Nombre,Descripcion
0,1,Electrónica,"Teléfonos, computadoras, accesorios y disposit..."
1,2,Moda,"Ropa, calzado, bolsos y accesorios para hombre..."
2,3,Hogar y Cocina,"Muebles, electrodomésticos, utensilios y decor..."
3,4,Deportes y Aire Libre,"Equipamiento, ropa deportiva y artículos para ..."
4,5,Belleza y Cuidado Personal,"Cosméticos, productos para el cabello, piel y ..."


,ProductoID,Nombre,Descripcion,Precio,Stock,CategoriaID
0,1,Smartphone Galaxy A54,Teléfono inteligente con pantalla AMOLED y cám...,349.99,50,1
1,2,Laptop Dell Inspiron 15,Laptop para trabajo y estudio con procesador I...,799.00,30,1
2,3,Auriculares Bluetooth Sony,Auriculares inalámbricos con cancelación de ru...,129.99,100,1
3,4,Camiseta Básica Hombre,Camiseta de algodón 100% disponible en varias ...,14.99,200,2
4,5,Jeans Skinny Mujer,Jeans corte skinny de mezclilla azul.,39.99,150,2


,UsuarioID,Nombre,Apellido,DNI,Email,Contraseña,FechaRegistro
0,1,Encarna,Donaire,49877134,encarna.donaire1@correo.com,Contraseña123,2025-07-23 01:22:40.282271
1,2,Jose Ignacio,Canales,98778810,jose ignacio.canales2@correo.com,Contraseña123,2025-07-23 01:22:40.282271
2,3,José,Sebastián,80095714,josé.sebastián3@correo.com,Contraseña123,2025-07-23 01:22:40.282271
3,4,Priscila,Vilar,94128750,priscila.vilar4@correo.com,Contraseña123,2025-07-23 01:22:40.282271
4,5,Epifanio,Ríos,62958342,epifanio.ríos5@correo.com,Contraseña123,2025-07-23 01:22:40.282271


,OrdenID,UsuarioID,FechaOrden,Total,Estado
0,1,153,2024-09-09 09:10:27,393.81,Enviado
1,2,810,2024-09-29 09:10:27,918.85,Completado
2,3,404,2024-11-03 09:10:27,391.29,Pendiente
3,4,52,2024-09-14 09:10:27,250.03,Cancelado
4,5,811,2025-04-11 09:10:27,983.13,Completado


,DetalleID,OrdenID,ProductoID,Cantidad,PrecioUnitario
0,1,833,33,1,425.26
1,2,657,21,4,307.13
2,3,911,15,4,444.75
3,4,818,26,5,346.24
4,5,695,12,3,155.05


...

,CarritoID,UsuarioID,ProductoID,Cantidad,FechaAgregado
0,1,844,22,3,2025-04-29 10:15:10
1,2,901,7,2,2025-05-14 10:15:10
2,3,824,28,1,2025-04-26 10:15:10
3,4,94,5,3,2025-04-20 10:15:10
4,5,228,15,3,2025-05-11 10:15:10


,OrdenMetodoID,OrdenID,MetodoPagoID,MontoPagado
0,1,1,2,580.22
1,2,2,4,638.87
2,3,3,7,946.09
3,4,4,2,390.35
4,5,5,3,908.67


,MetodoPagoID,Nombre,Descripcion
0,1,Tarjeta de Crédito,"Pago con tarjeta de crédito VISA, MasterCard o..."
1,2,Tarjeta de Débito,Pago con tarjeta de débito emitida por cualqui...
2,3,Transferencia Bancaria,Transferencia directa a nuestra cuenta bancaria
3,4,Pago Contra Entrega,Paga en efectivo cuando recibes tu pedido
4,5,Mercado Pago,Pago a través de la pasarela Mercado Pago


,ReseñaID,UsuarioID,ProductoID,Calificacion,Comentario,Fecha
0,1,898,10,1,"Mala calidad, no lo recomiendo. Ullam sint ali...",2025-07-23 01:23:07.755039
1,2,700,16,3,El producto es aceptable por el precio. Quisqu...,2025-07-23 01:23:07.755153
2,3,173,13,2,"Mala calidad, no lo recomiendo. Illum corporis...",2025-07-23 01:23:07.755191
3,4,557,20,2,No cumplió con lo prometido. Tempore distincti...,2025-07-23 01:23:07.755220
4,5,448,33,2,No cumplió con lo prometido. Modi doloribus re...,2025-07-23 01:23:07.755245


,PagoID,OrdenID,MetodoPagoID,Monto,FechaPago,EstadoPago
0,1,1,4,147.13,2025-07-23 01:23:16.081831,pending
1,2,2,1,647.98,2025-07-23 01:23:16.081936,pending
2,3,3,6,627.45,2025-07-23 01:23:16.081963,pending
3,4,4,5,848.07,2025-07-23 01:23:16.081982,pending
4,5,5,5,189.80,2025-07-23 01:23:16.082006,pending


## 2.Implementar análisis exploratorio complementario en Python mediante un ORM (como SQLAlchemy o psycopg2).

En [Archivo Entrega 1](1st_delivery.ipynb)
